In [ ]:
!pip install flask flask_cors tweepy transformers torch pandas matplotlib


In [ ]:
!pip install flask flask-cors tweepy transformers torch pandas matplotlib pyngrok
!ngrok config add-authtoken 2sJEuojAIXme0CuF7o0FH2DYhSK_2fBAphZTKf9uyXLWGvQRH


In [ ]:
import tweepy
import pandas as pd
import time
import json
import matplotlib.pyplot as plt
from transformers import pipeline
from flask import Flask, request, jsonify
from flask_cors import CORS

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Set pandas option to display full tweet text
pd.set_option('display.max_colwidth', None)

# Twitter API credentials (use secure storage in production!)
api_key = "hjuola1OWKXvXAalYGOcFA3rA"
api_key_secret = "NKdKURGjZsHmO6Woa0wyrhZC8w1qEhAl38lkypd7w8EFeaYBS5"
access_token = "1897656288533999616-e1CbS66X9dJBbkyjzXhPcecluLIIkU"
access_token_secret = "puTxOb8uRHt6Y6kEhfPRvpMqbaP5XvCkVaXBEybvTeB33"
bearer_token = "AAAAAAAAAAAAAAAAAAAAACcLzwEAAAAATe7LZ%2F2VWSGLF3%2FzNujN%2FSNhqQU%3DUJo6eA3eqj9Ibyhc66h7icH6p1YHXNrdPSA3InhAMLI01yBc7r"

# Create a Tweepy client for API v2
client = tweepy.Client(bearer_token=bearer_token,
                       consumer_key=api_key,
                       consumer_secret=api_key_secret,
                       access_token=access_token,
                       access_token_secret=access_token_secret)

# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
# Initialize the emotion detection pipeline
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)

# Function to convert the model's star rating into a sentiment category
def get_category(sentiment):
    label = sentiment['label']  # e.g., "5 stars"
    rating = int(label.split()[0])
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

@app.route('/analyze', methods=['POST'])
def analyze():
    try:
        data = request.get_json()
        user_query = data.get("query", "")
        if not user_query:
            return jsonify({"error": "No query provided"}), 400

        # Build the query string
        query = f"{user_query} -is:retweet lang:en"

        # Try to fetch recent tweets using Twitter API v2
        try:
            response = client.search_recent_tweets(query=query, max_results=10)
            tweets = response.data if response.data is not None else []
        except tweepy.TooManyRequests:
            return jsonify({"error": "Rate limit exceeded. Please wait and try again later."}), 429

        if len(tweets) < 10:
            print(f"Warning: Only {len(tweets)} tweet(s) were fetched. Expected minimum 10 tweets if available.")

        results = []

        # Analyze sentiment and emotion for each fetched tweet
        if tweets:
            for tweet in tweets:
                tweet_text = tweet.text
                # Sentiment analysis
                sentiment_result = sentiment_pipeline(tweet_text)[0]
                label = sentiment_result['label']
                rating = int(label.split()[0])
                category = get_category(sentiment_result)
                # Emotion detection
                emotion_result = emotion_pipeline(tweet_text)[0]
                emotion_label = emotion_result['label']
                emotion_score = emotion_result['score']
                results.append({
                    "Tweet": tweet_text,
                    "Rating": rating,
                    "Sentiment": category,
                    "Emotion": emotion_label,
                    "Emotion Score": round(emotion_score, 3)
                })

            # Return results as JSON
            return jsonify(results)

        else:
            return jsonify({"error": "No tweets found for the query or rate limit exceeded."}), 404

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Device set to use cpu
Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Mar/2025 14:38:06] "OPTIONS /analyze HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [06/Mar/2025 14:38:10] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Mar/2025 14:40:41] "OPTIONS /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Mar/2025 14:40:42] "POST /analyze HTTP/1.1" 429 -


In [4]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.connect(5000)


<NgrokTunnel: "https://e798-34-74-175-116.ngrok-free.app" -> "http://localhost:5000">